# <font color=blue>**Pandas II - Aplicações na Vida Real - FEA.dev**</font>

Após a aula introdutória de Pandas vamos nos aprofundar mais em algumas funcionalidades da biblioteca.

# <font color=blue>Sumário</font>

- ### 1. Introdução
        
        1.1 Apresentação das bases de dados
        1.2 Loc e iloc

- ### 2. Métodos Úteis
        
        2.1 Agrupamento
        2.1.1 Groupby
        2.1.2 Pivot Table
        2.1.3 Melt
        
        2.2.1 Diff
        2.2.2 Pct_change

- ### 3. Aplicações úteis

        3.1 Cálculo de retorno
        3.2 Melhores retornos para cada fundos
        3.3 Juntando os Resultados

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

## 1. Introdução

### 1.1 Apresentação das bases de dados

In [ ]:
ct = pd.read_csv("carteiras.csv", parse_dates=["data"])
at = pd.read_csv("ativos.csv", index_col=0, parse_dates=True)

In [ ]:
ct.info()

In [ ]:
ct.describe()

In [ ]:
ct.head()

In [ ]:
at.info()

In [ ]:
at.describe()

In [ ]:
at.head()

### Loc e iloc

In [ ]:
dv = pd.read_csv("dados_devers.csv", sep=";",
                 usecols=range(4),
                 index_col=0)

In [ ]:
dv.head()

In [ ]:
dv.loc["Eddy"]

In [ ]:
dv.loc["Bailinho"]

In [ ]:
dv.loc[["Eddy", "André"]]

In [ ]:
dv.loc[["Eddy", "Thais"], ["Geração", "Cursos 2022"]]

In [ ]:
dv.loc[["Eddy", "Thais"], "Cursos 2022":]

In [ ]:
# cells
dv.iloc[0]

In [ ]:
dv.iloc[0].name

In [ ]:
dv.iloc[range(2), range(1)]

In [ ]:
dv.iloc[[9, 5, 6]]

In [ ]:
del dv

In [ ]:
at.head()

In [ ]:
at.loc["2022-01-10":, :]

### Retorno a apresentação das bases

In [ ]:
# unique
ct['ativo'].unique()

In [ ]:
ct['ativo'].unique()[0]

In [ ]:
print(len(ct["ativo"].unique()), "vs", len(at.columns))
print(ct["ativo"].unique()[0])

#### Set Theory
![set_theory](set_theory.png)
![set_theory_ativos](set_theory_ativos.png)

In [ ]:
# sets https://www.w3schools.com/python/python_ref_set.asp
x = {'Petrobras', 'Itau', 'Suzano'}
y = {'Eletrobras', 'Bradesco', 'Suzano'}

z = x.difference(y)
z

In [ ]:
y.difference(x)

In [ ]:
x.symmetric_difference(y)

In [ ]:
x.intersection(y)

In [ ]:
# acessar colunas do dataframe de ativos
at.columns

In [ ]:
set(at.columns.unique()).difference(ct["ativo"].unique())

In [ ]:
set(ct["ativo"].unique()).difference(at.columns.unique())

In [ ]:
ct.loc[ct["ativo"].isin(at.columns), ["ativo", "ativo_desc"]].drop_duplicates()

In [ ]:
ct.loc[ct["ativo"].isin(at.columns), "ativo_desc"].unique()

In [ ]:
pd.set_option('max_colwidth', None)
ct[["nome_gestor", "nome_fundo"]].drop_duplicates().reset_index(drop=True)

In [ ]:
pd.reset_option('^display.', silent=True)

## 2. Métodos Úteis

### 2.1.1 Groupby

In [ ]:
ct.groupby("nome_fundo").nunique()[["ativo"]]

In [ ]:
ct[["nome_fundo", "data", "ativo"]].groupby(["nome_fundo", "data"]).count()

### 2.1.2 Pivot Table

In [ ]:
ct.pivot_table("ativo_desc", index="nome_fundo", columns="data", aggfunc=lambda x: len(x.unique()))

In [ ]:
ct.pivot_table("ativo", index="nome_fundo", columns="data", aggfunc=lambda x: len(x.unique()))

In [ ]:
pivoted_table = ct.pivot_table("ativo", index="nome_fundo", columns="data", aggfunc="count")
pivoted_table

### 2.1.3 Melt

In [ ]:
pivoted_table.melt(ignore_index=False).head(6)

### 2.2.1 Diff

In [ ]:
np.random.seed(420)
rand_series = np.random.rand(100)
plt.plot(np.arange(100), rand_series)

In [ ]:
x = np.arange(100).reshape(-1, 1)
y_tendencia = (rand_series * np.arange(1, 20, .19)).reshape(-1, 1)
pred = LinearRegression().fit(x, y_tendencia).predict(x)

In [ ]:
fix, ax = plt.subplots()
ax.plot(x, y_tendencia)
ax.plot(pred)
plt.show()

In [ ]:
y = pd.Series(y_tendencia.flatten()).diff()
pred = LinearRegression().fit(x, y.fillna(0)).predict(x)

fig, ax = plt.subplots()
# função plot do pandas
y.diff().plot()
ax.plot(pred)
plt.show()

### 2.2.2 Pct_change

In [ ]:
((at - at.shift(1)) / at).head()

In [ ]:
(at.diff() / at).head()

In [ ]:
at.pct_change().head()

In [ ]:
at.pct_change().iloc[1:]

In [ ]:
filling_first = at.pct_change()

In [ ]:
filling_first.iloc[0] = 0

In [ ]:
filling_first.head()

## 3. Aplicações Úteis

### 3.1 Cálculo de Retorno

In [ ]:
((at.pct_change() + 1).prod() - 1).sort_values(ascending=False)[:5]

In [ ]:
((at.pct_change() + 1).prod() - 1)[:3]

In [ ]:
top_10_geral = ((at.pct_change() + 1).prod() - 1).sort_values(ascending=False)[:10]

### 3.2 Cálculo Para Cada Fundo

In [ ]:
ativos_tarkus = (ct.loc[ct["ativo"].isin(at.columns) &
                        (ct["nome_fundo"]=="KAPITALO TARKUS MÁSTER FUNDO DE INVESTIMENTO EM AÇÕES"),
                        "ativo"]
                   .unique()
)
ativos_t3 = (ct["ativo"][ct["ativo"].isin(at.columns) &
                         (ct["nome_fundo"]=="T3 FUNDO DE INVESTIMENTO MULTIMERCADO CRÉDITO PRIVADO LONG BIASED")]
                   .unique()
)

In [ ]:
top_10_tarkus = ((at[ativos_tarkus].pct_change() + 1).prod() - 1).sort_values(ascending=False)[:10]
top_10_t3 = ((at[ativos_t3].pct_change() + 1).prod() - 1).sort_values(ascending=False)[:10]

### 3.3 Juntando os Resultados

In [ ]:
pd.concat([top_10_geral.rename("Geral"),
           top_10_tarkus.rename("Kapitalo Tarkus"),
           top_10_t3.rename("Tower Three Long Biased")], axis=1)

![](joins.png)

In [ ]:
(pd.DataFrame(top_10_geral.rename("Geral"))
   .merge(top_10_tarkus.rename("Tarkus"),
          left_index=True,
          right_index=True,
          how="outer")
   .merge(top_10_t3.rename("Tower Three Long Biased"),
          left_index=True,
          right_index=True,
          how="outer")
).sort_values("Geral", ascending=False)

In [ ]:
(pd.DataFrame(top_10_geral.rename("Geral"))
   .join(top_10_tarkus.rename("Tarkus"),
          how="outer")
   .join(top_10_t3.rename("Tower Three Long Biased"),
          how="outer")
).sort_values("Geral", ascending=False)

#### **Bônus:** Vídeo sobre cálculo de retornos FEA.dev

https://www.youtube.com/watch?v=8FjerLRybkg&t=5s